In [11]:
import pdfplumber
import json_repair
import pandas as pd
def extract_text_from_pdf(pdf_path):
    txt = ''
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            txt += page.extract_text()
    return txt

In [12]:
from gradio_client import Client

client = Client("Qwen/Qwen2-72B-Instruct")

def chatbot(prompt,input_text):
    result = client.predict(
        query=input_text,
        history=[],
        system=prompt,
        api_name="/model_chat"
    )
    return result[1][0][1]


Loaded as API: https://qwen-qwen2-72b-instruct.hf.space ✔


In [13]:
prompt = '''请帮我从下面这篇周报中总结作者读的论文。其中write_date为date_range的最后一天,除了论文标题和期刊名称用英文外，总结用中文。返回格式：
```json
{
    date_range: "yyyy.mm.dd-yyyy.mm.dd",
    write_date: "yyyy-mm-dd",
    papers: [
        {
            "title": "论文名称",
            "journal": "期刊名称",
            "content": "一句话总结",
            "conclusion": "一句话总结",
            "gain": "一句话总结"
        },
        {
            "title": "论文名称",
            "journal": "期刊名称",
            "content": "一句话总结",
            "conclusion": "一句话总结",
            "gain": "一句话总结"
        }
    ]
}
```
'''

In [14]:
def summarize_weekly(result,new_name):
    paperstr = pd.DataFrame(result['papers']).to_markdown(index=False)

    mdstr = f'''---
title: '周小结({result["date_range"]})'
date: {result["write_date"]}
permalink: /posts/{result["write_date"]}_week/
---
{paperstr}

<embed src="/{new_name}" type="application/pdf" height="400px" />
    '''
    return mdstr

In [15]:
from glob import glob
import os
import shutil
import json

for pdf in glob("files/post1/*.pdf"):
    text = extract_text_from_pdf(pdf)
    result = chatbot(prompt,text)
    result = json_repair.loads(result)
    new_name = "files/post/" + result["write_date"] + "-week.pdf"
    os.rename(pdf,new_name)
    summarize_path = "_posts/" + result["write_date"] + "-week.md"
    summarize = summarize_weekly(result,new_name)
    with open(summarize_path,"w") as f:
        f.write(summarize)

    # json.dump(result,open("files/summary/" + result["write_date"] + "-week.json",'w'))
    
